In [1]:
CUDA_LAUNCH_BLOCKING=1

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import warnings
import copy
#
import os
import torch
import numpy as np


class Graph:
    """The Graph to model the skeletons extracted by the Alpha-Pose.
    Args:
        - strategy: (string) must be one of the follow candidates
            - uniform: Uniform Labeling,
            - distance: Distance Partitioning,
            - spatial: Spatial Configuration,
        For more information, please refer to the section 'Partition Strategies'
            in our paper (https://arxiv.org/abs/1801.07455).
        - layout: (string) must be one of the follow candidates
            - coco_cut: Is COCO format but cut 4 joints (L-R ears, L-R eyes) out.
        - max_hop: (int) the maximal distance between two connected nodes.
        - dilation: (int) controls the spacing between the kernel points.
    """
    def __init__(self,
                 layout='coco_cut',
                 strategy='uniform',
                 max_hop=1,
                 dilation=1):
        self.max_hop = max_hop
        self.dilation = dilation

        self.get_edge(layout)
        self.hop_dis = get_hop_distance(self.num_node, self.edge, max_hop)
        self.get_adjacency(strategy)

    def get_edge(self, layout):
        if layout == 'coco_cut':
            self.num_node = 14
            self_link = [(i, i) for i in range(self.num_node)]
            neighbor_link = [(6, 4), (4, 2), (2, 13), (13, 1), (5, 3), (3, 1), (12, 10),
                             (10, 8), (8, 2), (11, 9), (9, 7), (7, 1), (13, 0)]
            self.edge = self_link + neighbor_link
            self.center = 13
        else:
            raise ValueError('This layout is not supported!')

    def get_adjacency(self, strategy):
        valid_hop = range(0, self.max_hop + 1, self.dilation)
        adjacency = np.zeros((self.num_node, self.num_node))
        for hop in valid_hop:
            adjacency[self.hop_dis == hop] = 1
        normalize_adjacency = normalize_digraph(adjacency)

        if strategy == 'uniform':
            A = np.zeros((1, self.num_node, self.num_node))
            A[0] = normalize_adjacency
            self.A = A
        elif strategy == 'distance':
            A = np.zeros((len(valid_hop), self.num_node, self.num_node))
            for i, hop in enumerate(valid_hop):
                A[i][self.hop_dis == hop] = normalize_adjacency[self.hop_dis ==
                                                                hop]
            self.A = A
        elif strategy == 'spatial':
            A = []
            for hop in valid_hop:
                a_root = np.zeros((self.num_node, self.num_node))
                a_close = np.zeros((self.num_node, self.num_node))
                a_further = np.zeros((self.num_node, self.num_node))
                for i in range(self.num_node):
                    for j in range(self.num_node):
                        if self.hop_dis[j, i] == hop:
                            if self.hop_dis[j, self.center] == self.hop_dis[i, self.center]:
                                a_root[j, i] = normalize_adjacency[j, i]
                            elif self.hop_dis[j, self.center] > self.hop_dis[i, self.center]:
                                a_close[j, i] = normalize_adjacency[j, i]
                            else:
                                a_further[j, i] = normalize_adjacency[j, i]
                if hop == 0:
                    A.append(a_root)
                else:
                    A.append(a_root + a_close)
                    A.append(a_further)
            A = np.stack(A)
            self.A = A
            #self.A = np.swapaxes(np.swapaxes(A, 0, 1), 1, 2)
        else:
            raise ValueError("This strategy is not supported!")


def get_hop_distance(num_node, edge, max_hop=1):
    A = np.zeros((num_node, num_node))
    for i, j in edge:
        A[j, i] = 1
        A[i, j] = 1

    # compute hop steps
    hop_dis = np.zeros((num_node, num_node)) + np.inf
    transfer_mat = [np.linalg.matrix_power(A, d) for d in range(max_hop + 1)]
    arrive_mat = (np.stack(transfer_mat) > 0)
    for d in range(max_hop, -1, -1):
        hop_dis[arrive_mat[d]] = d
    return hop_dis


def normalize_digraph(A):
    Dl = np.sum(A, 0)
    num_node = A.shape[0]
    Dn = np.zeros((num_node, num_node))
    for i in range(num_node):
        if Dl[i] > 0:
            Dn[i, i] = Dl[i]**(-1)
    AD = np.dot(A, Dn)
    return AD


def normalize_undigraph(A):
    Dl = np.sum(A, 0)
    num_node = A.shape[0]
    Dn = np.zeros((num_node, num_node))
    for i in range(num_node):
        if Dl[i] > 0:
            Dn[i, i] = Dl[i]**(-0.5)
    DAD = np.dot(np.dot(Dn, A), Dn)
    return DAD

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class GraphConvolution(nn.Module):
    """The basic module for applying a graph convolution.
    Args:
        - in_channel: (int) Number of channels in the input sequence data.
        - out_channels: (int) Number of channels produced by the convolution.
        - kernel_size: (int) Size of the graph convolving kernel.
        - t_kernel_size: (int) Size of the temporal convolving kernel.
        - t_stride: (int, optional) Stride of the temporal convolution. Default: 1
        - t_padding: (int, optional) Temporal zero-padding added to both sides of
            the input. Default: 0
        - t_dilation: (int, optional) Spacing between temporal kernel elements. Default: 1
        - bias: (bool, optional) If `True`, adds a learnable bias to the output.
            Default: `True`
    Shape:
        - Inputs x: Graph sequence in :math:`(N, in_channels, T_{in}, V)`,
                 A: Graph adjacency matrix in :math:`(K, V, V)`,
        - Output: Graph sequence out in :math:`(N, out_channels, T_{out}, V)`

            where
                :math:`N` is a batch size,
                :math:`K` is the spatial kernel size, as :math:`K == kernel_size[1]`,
                :math:`T_{in}/T_{out}` is a length of input/output sequence,
                :math:`V` is the number of graph nodes.

    """
    def __init__(self, in_channels, out_channels, kernel_size,
                 t_kernel_size=1,
                 t_stride=1,
                 t_padding=0,
                 t_dilation=1,
                 bias=True):
        super().__init__()

        self.kernel_size = kernel_size
        self.conv = nn.Conv2d(in_channels,
                              out_channels * kernel_size,
                              kernel_size=(t_kernel_size, 1),
                              padding=(t_padding, 0),
                              stride=(t_stride, 1),
                              dilation=(t_dilation, 1),
                              bias=bias)

    def forward(self, x, A):
        x = self.conv(x)
        n, kc, t, v = x.size()
        x = x.view(n, self.kernel_size, kc//self.kernel_size, t, v)
        x = torch.einsum('nkctv,kvw->nctw', (x, A))

        return x.contiguous()
###########################################################
class Channel_Attention(nn.Module):
    def __init__(self,out_channels):
        super().__init__()
        
        self.atten = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Conv2d(out_channels,int(out_channels/4),1),
            nn.BatchNorm2d(int(out_channels/4)),
            nn.ReLU(),
            nn.Conv2d(int(out_channels/4),out_channels,1),
            nn.Sigmoid(),
        )
    def forward(self,x):
        
        atten = self.atten(x)
        # print(x.shape,atten.shape)
        x = x*atten
        
        return x
    

###########################################################

class st_gcn(nn.Module):
    """Applies a spatial temporal graph convolution over an input graph sequence.
    Args:
        - in_channels: (int) Number of channels in the input sequence data.
        - out_channels: (int) Number of channels produced by the convolution.
        - kernel_size: (tuple) Size of the temporal convolving kernel and
            graph convolving kernel.
        - stride: (int, optional) Stride of the temporal convolution. Default: 1
        - dropout: (int, optional) Dropout rate of the final output. Default: 0
        - residual: (bool, optional) If `True`, applies a residual mechanism.
            Default: `True`
    Shape:
        - Inputs x: Graph sequence in :math: `(N, in_channels, T_{in}, V)`,
                 A: Graph Adjecency matrix in :math: `(K, V, V)`,
        - Output: Graph sequence out in :math: `(N, out_channels, T_{out}, V)`
            where
                :math:`N` is a batch size,
                :math:`K` is the spatial kernel size, as :math:`K == kernel_size[1]`,
                :math:`T_{in}/T_{out}` is a length of input/output sequence,
                :math:`V` is the number of graph nodes.
    """
    def __init__(self, in_channels, out_channels, kernel_size,
                 stride=1,
                 dropout=0,
                 residual=True):
        super().__init__()
        assert len(kernel_size) == 2
        assert kernel_size[0] % 2 == 1
        #print(kernel_size)(9, 3)
        padding = ((kernel_size[0] - 1) // 2, 0)
        #print(padding)(4, 0)

        self.gcn = GraphConvolution(in_channels, out_channels, kernel_size[1])
        self.tcn = nn.Sequential(nn.BatchNorm2d(out_channels),
                                 nn.ReLU(inplace=False),
                                 nn.Conv2d(out_channels,
                                           out_channels,
                                           (kernel_size[0], 1),
                                           (stride, 1),
                                           padding),
                                 nn.BatchNorm2d(out_channels),
                                 nn.Dropout(dropout, inplace=True),
                                 )

        if not residual:
            self.residual = lambda x: 0
        elif (in_channels == out_channels) and (stride == 1):
            self.residual = lambda x: x
        else:
            self.residual = nn.Sequential(nn.Conv2d(in_channels,
                                                    out_channels,
                                                    kernel_size=1,
                                                    stride=(stride, 1)),
                                          nn.BatchNorm2d(out_channels)
                                          )
        self.relu = nn.ReLU(inplace=False)
        self.channel_attention_module = Channel_Attention(out_channels) 
        
    def forward(self, x, A):
        res = self.residual(x)
        # print(res)
        x = self.gcn(x, A)
        #print("x_in:",x.size())
        # x = self.tcn(x) + res
        #print("x_out:",x.size())
        x = self.tcn(x)
        x = self.channel_attention_module(x) + res
        
        return self.relu(x)


class StreamSpatialTemporalGraph(nn.Module):
    """Spatial temporal graph convolutional networks.
    Args:
        - in_channels: (int) Number of input channels.
        - graph_args: (dict) Args map of `Actionsrecognition.Utils.Graph` Class.
        - num_class: (int) Number of class outputs. If `None` return pooling features of
            the last st-gcn layer instead.
        - edge_importance_weighting: (bool) If `True`, adds a learnable importance
            weighting to the edges of the graph.
        - **kwargs: (optional) Other parameters for graph convolution units.
    Shape:
        - Input: :math:`(N, in_channels, T_{in}, V_{in})`
        - Output: :math:`(N, num_class)` where
            :math:`N` is a batch size,
            :math:`T_{in}` is a length of input sequence,
            :math:`V_{in}` is the number of graph nodes,
        or If num_class is `None`: `(N, out_channels)`
            :math:`out_channels` is number of out_channels of the last layer.
    """
    def __init__(self, in_channels, graph_args, num_class=None,
                 edge_importance_weighting=True, **kwargs):
        super().__init__()
        # Load graph.
        graph = Graph(**graph_args)
        A = torch.tensor(graph.A, dtype=torch.float32, requires_grad=False)
        self.register_buffer('A', A)

        # Networks.
        spatial_kernel_size = A.size(0)
        temporal_kernel_size = 9
        kernel_size = (temporal_kernel_size, spatial_kernel_size)
        kwargs0 = {k: v for k, v in kwargs.items() if k != 'dropout'}

        self.data_bn = nn.BatchNorm1d(in_channels * A.size(1))
        self.st_gcn_networks = nn.ModuleList((
            st_gcn(in_channels, 64, kernel_size, 1, residual=False, **kwargs0),
            
            st_gcn(64, 64, kernel_size, 1, **kwargs),
            st_gcn(64, 64, kernel_size, 1, **kwargs),
            # st_gcn(64, 64, kernel_size, 1, **kwargs),
            st_gcn(64, 128, kernel_size, 2, **kwargs),
            st_gcn(128, 128, kernel_size, 1, **kwargs),
            # st_gcn(128, 128, kernel_size, 1, **kwargs),
            st_gcn(128, 256, kernel_size, 2, **kwargs),
            st_gcn(256, 256, kernel_size, 1, **kwargs),
            # st_gcn(256, 256, kernel_size, 1, **kwargs)
        ))

        # initialize parameters for edge importance weighting.
        if edge_importance_weighting:
            self.edge_importance = nn.ParameterList([
                nn.Parameter(torch.ones(A.size()))
                for i in self.st_gcn_networks
            ])
        else:
            self.edge_importance = [1] * len(self.st_gcn_networks)

        if num_class is not None:
            self.cls = nn.Conv2d(256, num_class, kernel_size=1)
        else:
            self.cls = lambda x: x

    def forward(self, x):
        # data normalization.
        N, C, T, V = x.size()
        x = x.permute(0, 3, 1, 2).contiguous()  # (N, V, C, T)
        x = x.view(N, V * C, T)
        x = self.data_bn(x)
        x = x.view(N, V, C, T)
        x = x.permute(0, 2, 3, 1).contiguous()
        x = x.view(N, C, T, V)

        # forward.
        for gcn, importance in zip(self.st_gcn_networks, self.edge_importance):
            x = gcn(x, self.A * importance)
        #(B: 32,256, 6 or 7, node: 14, )
        x = F.avg_pool2d(x, x.size()[2:])
        x = self.cls(x)
        x = x.view(x.size(0), -1)

        return x


class TwoStreamSpatialTemporalGraph(nn.Module):
    """Two inputs spatial temporal graph convolutional networks.
    Args:
        - graph_args: (dict) Args map of `Actionsrecognition.Utils.Graph` Class.
        - num_class: (int) Number of class outputs.
        - edge_importance_weighting: (bool) If `True`, adds a learnable importance
            weighting to the edges of the graph.
        - **kwargs: (optional) Other parameters for graph convolution units.
    Shape:
        - Input: :tuple of math:`((N, 3, T, V), (N, 2, T, V))`
        for points and motions stream where.
            :math:`N` is a batch size,
            :math:`in_channels` is data channels (3 is (x, y, score)), (2 is (mot_x, mot_y))
            :math:`T` is a length of input sequence,
            :math:`V` is the number of graph nodes,
        - Output: :math:`(N, num_class)`
    """
    def __init__(self, graph_args, num_class, edge_importance_weighting=True,
                 **kwargs):
        super().__init__()
        self.pts_stream = StreamSpatialTemporalGraph(3, graph_args, None,
                                                     edge_importance_weighting,
                                                     **kwargs)#3 is x,y,z
        self.mot_stream = StreamSpatialTemporalGraph(2, graph_args, None,
                                                     edge_importance_weighting,
                                                     **kwargs)#2 is x,y
        
        # self.model = BiLSTM(input_size=4,hidden_size=64,num_layers=1,dropout_prob=0.3,num_classes=2,feature="mean").to(device)
        
        self.fcn = nn.Linear(256 * 2, num_class)
        # self.fcn = nn.Linear((256 * 2)+11, num_class)
        # print("errr")
        
        
        # self.attention1=nn.Sequential(
        #     nn.Linear(256 * 2, 128),
        #     nn.BatchNorm1d(128),
        #     nn.LeakyReLU(inplace=False),
        #     nn.Linear(128, 256 * 2),
        #     nn.BatchNorm1d(256 * 2),
        #     nn.Sigmoid(),
        # )
        
        # self.attention2=nn.Sequential(
        #     nn.Linear(256 * 2, 128),
        #     nn.BatchNorm1d(128),
        #     nn.ReLU(inplace=False),
        #     nn.Linear(128, 256 * 2),
        #     nn.BatchNorm1d(256 * 2),
        #     nn.Sigmoid(),
        # )
        
        # self.attention3=nn.Sequential(
        #     nn.Linear(256 * 2, 64),
        #     nn.BatchNorm1d(64),
        #     nn.ReLU(inplace=False),
        #     nn.Linear(64, 256 * 2),
        #     nn.BatchNorm1d(256 * 2),
        #     nn.Sigmoid(),
        # )
        
        # self.attention4=nn.Sequential(
        #     nn.Linear(256 * 2, 64),
        #     nn.BatchNorm1d(64),
        #     nn.ReLU(inplace=False),
        #     nn.Linear(64, 64),
        #     nn.BatchNorm1d(64),
        #     nn.ReLU(inplace=False),
        #     nn.Linear(64, 256 * 2),
        #     nn.BatchNorm1d(256 * 2),
        #     nn.Sigmoid(),
        # )
        
        # self.attention5=nn.Sequential(
        #     nn.Linear(256 * 2, 128),
        #     nn.BatchNorm1d(128),
        #     nn.GELU(),
        #     nn.Linear(128, 256 * 2),
        #     nn.BatchNorm1d(256 * 2),
        #     nn.Sigmoid(),
        # )
        
    def forward(self, inputs):
        out1 = self.pts_stream(inputs[0])
        out2 = self.mot_stream(inputs[1])
        
        # out3 = self.model(input[3])
        
        #print(out1.size())torch.Size([32, 256])
        #print(out2.size())torch.Size([32, 256])
        concat = torch.cat([out1, out2], dim=-1)
        # concat = torch.cat([out1, out2, out3], dim=-1)
        
        # concat =  self.attention1(concat)
        # concat =  self.attention2(concat)
        # concat =  self.attention3(concat)
        # concat =  self.attention4(concat)
        # concat =  self.attention5(concat)
        
        out = self.fcn(concat)
        #print(out.size())
        # return torch.sigmoid(out)
        return F.softmax(out,dim=-1)

/home/egawa/anaconda3/envs/pt113/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as f

####1DCNN(時系列により)######
class CNN1D(nn.Module):
    def __init__(self):
        super(CNN1D, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv1d(15, 16, kernel_size=5, padding=2), #batch,チャンネル、シーケンス 入力チャンネル=30, 出力チャンネル=16
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(2))
        self.layer2 = nn.Sequential(
            nn.Conv1d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2))
        self.fc = nn.Linear(32*7, 32)  

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)#torch.Size([32, 32, 7])
        # x = x.view(x.size(0), -1)  # Flatten
        # x = self.fc(x)torch.Size([32, 32])
        # print(x.size())
        return x
####LSTM#####
class ChannelAttention(nn.Module):
    def __init__(self, input_size, reduce_rate = 1/8):
        super(ChannelAttention,self).__init__()
        
        self.attention = nn.Sequential(
            nn.Linear(input_size, int(input_size*reduce_rate)),
            nn.ReLU(),
            nn.Linear(int(input_size*reduce_rate), input_size),
            nn.Sigmoid(),
        )
        
    def forward(self,x):
        weight = self.attention(x)
        x = torch.einsum('bc,bc -> bc',(x,weight))
        return x
    
class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout_prob, num_classes = 1, feature = "last"):
        super(BiLSTM,self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_classes = num_classes
        # Bidirectional LSTM layer
        self.lstm1 = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True,dropout=dropout_prob)
        
        self.batchnorm = nn.BatchNorm1d(hidden_size*2)
        self.channelattention = ChannelAttention(hidden_size*2)
        
        self.feature = feature
        
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(hidden_size*2,num_classes)
        )
        
    def forward(self, x):
        # x = self.attention(x)
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)  # Multiply by 2 for bidirectional
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate LSTM
        out, _ = self.lstm1(x, (h0, c0))
        
        
        # out = self.meanovertime(out)
        if self.feature=="last":
            out = out[:, -1, :]
        else:
            out = torch.mean(out,axis=1,keepdim=False)
        out = self.batchnorm(out)
        out = self.channelattention(out)
        out = self.fc(out)
        return out
    
class CNN_BiLSTM(nn.Module):
    def __init__(self, hidden_size, num_layers, dropout_prob, num_classes=1, feature="last"):
        super(CNN_BiLSTM, self).__init__()
        self.cnn = CNN1D()
        self.bilstm = BiLSTM(input_size=32,hidden_size=64,num_layers=1,dropout_prob=0.3,num_classes=11,feature="mean")
    
    def forward(self, x):
        # print(x.size())
        x = x.permute(0,2,1)
        cnn_out = self.cnn(x)
        # print(cnn_out)
        # cnn_out = cnn_out.unsqueeze(1)# LSTMに渡すために次元を調整 (batch, seq_len, feature)torch.Size([32, 1, 32])
        # print(cnn_out.size())
        cnn_out = cnn_out.permute(0,2,1)
        lstm_out = self.bilstm(cnn_out)
        return lstm_out
    
    
# model = BiLSTM(input_size=4,hidden_size=125,num_layers=1,dropout_prob=0.3,num_classes=2,feature="mean")

In [4]:
# inp=torch.randn(8,2,21,14)
# inp=inp.cuda()

# #print(a[1].shape)
# a=model(inp)
# print(a.shape)

In [5]:
import torch
from torch.utils.data import Dataset

class Skeleton_Sensor_Dataset(Dataset):
    def __init__(self,skeleton_sensor_data,label):
        self.skeleton_sensor_data = skeleton_sensor_data
        self.label = label
        
    def __len__(self):
        return len(self.skeleton_sensor_data)
    
    def __getitem__(self,idx):
        skeleton,sensor = self.skeleton_sensor_data[idx]
        label = self.label[idx]
        
        # print(f"DEBUG {type(skeleton.dtype)}, {type(sensor.dtype)} {type(label.dtype)} ")
        
        if not isinstance(skeleton,torch.Tensor):
            skeleton = torch.tensor(skeleton,dtype=torch.float32)
            
        if not isinstance(sensor,torch.Tensor):
            sensor = torch.tensor(sensor,dtype=torch.float32)
        
        if not isinstance(label,torch.Tensor):
            label = torch.tensor(label,dtype=torch.float32)
            
        skeleton = skeleton.permute(2,0,1)
        return skeleton,sensor,label
    
class Skeleton_Sensor_Dataset_v2(Dataset):
    def __init__(self,skeleton_sensor_data,label):
        self.skeleton_sensor_data = skeleton_sensor_data
        self.label = label
        
    def __len__(self):
        return len(self.skeleton_sensor_data)
    
    def __getitem__(self,idx):
        skeleton,sensor = self.skeleton_sensor_data[idx]
        label = self.label[idx]
        
        # print(f"DEBUG {type(skeleton.dtype)}, {type(sensor.dtype)} {type(label.dtype)} ")
        
        if not isinstance(skeleton,torch.Tensor):
            skeleton = torch.tensor(skeleton,dtype=torch.float32)
            
        if not isinstance(sensor,torch.Tensor):
            sensor = torch.tensor(sensor,dtype=torch.float32)
        
        if not isinstance(label,torch.Tensor):
            label = torch.tensor(label,dtype=torch.float32)
            
        skeleton = skeleton.permute(2,0,1) # skeleton [num_sample, (time, vertex, xyz)] -> [num_sample, (xyz, time, vertex)]
        return skeleton,sensor,label
    
def custom_collate_fn(batch):
    skeletons = [item[0] for item in batch]
    sensors = [item[1] for item in batch]
    labels = [item[2] for item in batch]

    skeletons = torch.stack(skeletons)
    sensors = torch.stack(sensors)
    labels = torch.stack(labels)  # 例: ラベルが整数である場合

    return skeletons, sensors, labels

In [6]:
def KFold_load_dataset(data_files, batch_size, split_size=0.2):#0.2
    """Load data files into torch DataLoader with/without spliting train-test.
    """
    features, labels = [], []
    for fil in data_files:
        with open(fil, 'rb') as f:
            fts, lbs = pickle.load(f)
            features.append(fts)
            labels.append(lbs)
        del fts, lbs
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    print(features.shape)
    print(labels.shape)
    
    set_of_train_loader,set_of_valid_loader = [],[]

    if split_size > 0:
        if split_size < 1:
            n_splits = int(1/split_size)
        else:
            n_splits = split_size
        skf = StratifiedKFold(n_splits,random_state=42, shuffle=True)
        
        for train_index, test_index in skf.split(features,labels):
            train_set = data.TensorDataset(torch.tensor(features[train_index], dtype=torch.float32),torch.tensor(labels[train_index], dtype=torch.int64))
            valid_set = data.TensorDataset(torch.tensor(features[test_index], dtype=torch.float32),torch.tensor(labels[test_index], dtype=torch.int64))
            train_loader = data.DataLoader(train_set, batch_size, shuffle=True)
            valid_loader = data.DataLoader(valid_set, batch_size)
            
            set_of_train_loader.append(train_loader)
            set_of_valid_loader.append(valid_loader)
            
    return set_of_train_loader, set_of_valid_loader

def KFold_load_dataset_v2(data_files, batch_size, split_size=0.2):
    """Load data files into torch DataLoader with K-Fold splitting."""
    videos = []
    features, sensors, labels = [], [], []
    
    # データの読み込み
    for fil in data_files:
        with open(fil, 'rb') as f:
            vid, fts, sr, lbs = pickle.load(f)
            videos += vid
            features.append(fts)
            sensors.append(sr)
            labels.append(lbs)
        del fts, lbs, sr
    
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    sensors = np.concatenate(sensors, axis=0)
    
    labels = labels.astype(np.float32)  # データ型の変換

    set_of_train_loader, set_of_valid_loader = [], []

    if split_size > 0:
        if split_size < 1:
            n_splits = int(1 / split_size)
        else:
            n_splits = split_size
        
        # skf = StratifiedKFold(n_splits, random_state=42, shuffle=True)
        skf = KFold(n_splits, random_state=42, shuffle=True)
        for train_index, test_index in skf.split(features, labels):
            train_samples = [(features[i], sensors[i]) for i in train_index]
            valid_samples = [(features[i], sensors[i]) for i in test_index]
            
            train_set = Skeleton_Sensor_Dataset(train_samples, labels[train_index])
            valid_set = Skeleton_Sensor_Dataset(valid_samples, labels[test_index])
            
            train_loader = data.DataLoader(train_set, batch_size, shuffle=True)
            valid_loader = data.DataLoader(valid_set, batch_size)
            
            set_of_train_loader.append(train_loader)
            set_of_valid_loader.append(valid_loader)
    
    return set_of_train_loader, set_of_valid_loader


In [7]:
def load_dataset(data_files, batch_size, random_state, split_size=0.3):#0.2
    """Load data files into torch DataLoader with/without spliting train-test.
    """
    features,sensors, labels = [], [], []
    for fil in data_files:
        # print(fil)
        with open(fil, 'rb') as f:
            _ ,fts,sr,lbs = pickle.load(f)
            # videos += vid
            features.append(fts)
            sensors.append(sr)
            labels.append(lbs)
        del fts, lbs,sr
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    sensors = np.concatenate(sensors, axis=0)
    # print(features.shape,labels.shape)
    # print(f"DEBUG {features.shape},{sensors.shape},{labels.shape}")
    labels = labels.astype(np.float32)
    
    samples = []
    for feature,sensor in zip(features,sensors):
        samples += [(feature,sensor)]
    # samples = np.array(samples,dtype=object)
    
     # 乱数シードを生成
    # random_state = np.random.randint(0, 100)
    print(f"Random State Used for Split: {random_state}")
    
    if split_size > 0:
        x_train, x_valid, y_train, y_valid = train_test_split(samples, labels, test_size=split_size,random_state=random_state)
        
        # train_set = data.TensorDataset(torch.tensor(x_train, dtype=torch.float32).permute(0, 3, 1, 2),
        #                                torch.tensor(y_train, dtype=torch.float32))
        # valid_set = data.TensorDataset(torch.tensor(x_valid, dtype=torch.float32).permute(0, 3, 1, 2),
        #                                torch.tensor(y_valid, dtype=torch.float32))
        train_set = Skeleton_Sensor_Dataset(x_train,y_train)
        valid_set = Skeleton_Sensor_Dataset(x_valid,y_valid)
        train_loader = data.DataLoader(train_set, batch_size, shuffle=True)
        valid_loader = data.DataLoader(valid_set, batch_size)
    else:
        # train_set = data.TensorDataset(torch.tensor(features, dtype=torch.float32).permute(0, 3, 1, 2),
        #                                torch.tensor(labels, dtype=torch.float32))
        train_set = Skeleton_Sensor_Dataset(features,labels)
        train_loader = data.DataLoader(train_set, batch_size, shuffle=True)
        valid_loader = None
    return train_loader, valid_loader


def load_dataset_v2(data_files, batch_size, random_state, split_size=0.3):#0.2
    """Load data files into torch DataLoader with/without spliting train-test.
    """
    videos = []
    features,sensors, labels = [], [], []
    for fil in data_files:
        # print(fil)
        with open(fil, 'rb') as f:
            vid ,fts,sr, lbs = pickle.load(f)
            videos += vid
            features.append(fts)
            sensors.append(sr)
            labels.append(lbs)
        del fts, lbs,sr
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    sensors = np.concatenate(sensors, axis=0)
    # print(features.shape,labels.shape)
    # print(f"DEBUG {features.shape},{sensors.shape},{labels.shape}")
    labels = labels.astype(np.float32) # dtype : object -> float32
    
    # random_state = np.random.randint(0, 100)
    print(f"Random State Used for Split: {random_state}")
    
    if split_size > 0:
        unique_video_names = np.unique(videos)
        train_videos, test_videos= train_test_split(unique_video_names,test_size=split_size,random_state=random_state)

        train_samples,valid_samples = [],[]
        train_label,valid_label = [],[]
        for video,feature,sensor,label in zip(videos,features,sensors,labels):
            if video in train_videos:
                train_samples += [(feature,sensor)]
                train_label += [label]
            else:
                valid_samples += [(feature,sensor)]
                valid_label += [label]
        
        train_set = Skeleton_Sensor_Dataset(train_samples,train_label)
        valid_set = Skeleton_Sensor_Dataset(valid_samples,valid_label)
        train_loader = data.DataLoader(train_set, batch_size, shuffle=True,)
        valid_loader = data.DataLoader(valid_set, batch_size)
    
    else:
        samples = []
        for feature,sensor in zip(features,sensors):
            samples += [(feature,sensor)]
        train_set = Skeleton_Sensor_Dataset(samples,labels)
        train_loader = data.DataLoader(train_set, batch_size, shuffle=True)
        valid_loader = None
    return train_loader, valid_loader

In [ ]:
#run the code
import os
import time
import torch
import pickle
import numpy as np
import torch.nn.functional as F
from shutil import copyfile
from tqdm import tqdm
from torch.utils import data
from torch.optim.adadelta import Adadelta
from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn import metrics

import copy
import seaborn as sns
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt

torch.autograd.set_detect_anomaly(False)
#二分割できる（訓練用とテスト用）

#from pose_utils import motions_map
import sys
sys.path.append("../")
# from Actionsrecognition.Models import *

from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
    KFold,
)

import pandas as pd
save_folder = 'saved/HAR_sensor(conv)_load_datasetv2(7:3)_11'

#device = 'cuda'
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("using", device, "device")

epochs = 100
batch_size = 32 #32

# DATA FILES.
# Should be in format of
#  inputs: (N_samples, time_steps, graph_node, channels),
#  labels: (N_samples, num_class)
#   and do some of normalizations on it. Default data create from:
#       Data.create_dataset_(1-3).py
# where
#   time_steps: Number of frame input sequence, Default: 30
#   graph_node: Number of node in skeleton, Default: 14
#   channels: Inputs data (x, y and scores), Default: 3
#   num_class: Number of pose class to train, Default: 7

data_files = ['../Data_fall2/har30_1_sensor_new-set(labelXscrw).pkl',
              '../Data_fall2/har30_2_sensor_new-set(labelXscrw).pkl',
              '../Data_fall2/har30_3_sensor_new-set(labelXscrw).pkl',
              '../Data_fall2/har30_4_sensor_new-set(labelXscrw).pkl',
              # '../Data_fall2/har30_5_sensor_new-set(labelXscrw).pkl',]
              '../Data_fall2/har30_6_sensor_new-set(labelXscrw).pkl',
              '../Data_fall2/har30_7_sensor_new-set(labelXscrw).pkl',
              '../Data_fall2/har30_8_sensor_new-set(labelXscrw).pkl',
              # '../Data_fall2/har30_9_sensor_new-set(labelXscrw).pkl',]
              '../Data_fall2/har30_10_sensor_new-set(labelXscrw).pkl',
              '../Data_fall2/har30_11_sensor_new-set(labelXscrw).pkl',
              '../Data_fall2/har30_12_sensor_new-set(labelXscrw).pkl',
              '../Data_fall2/har30_13_sensor_new-set(labelXscrw).pkl',
              '../Data_fall2/har30_14_sensor_new-set(labelXscrw).pkl',
              '../Data_fall2/har30_15_sensor_new-set(labelXscrw).pkl',
              '../Data_fall2/har30_16_sensor_new-set(labelXscrw).pkl',
              '../Data_fall2/har30_17_sensor_new-set(labelXscrw).pkl']

class_names = ['Falling_forwards_hands','Falling_forwards_knees','Falling_backwards','Falling_sidewards',
              'Falling_sitting','Walking','Standing','Sitting','Picking','Jumping','Laying']
# num_class = len(class_names)


def accuracy_batch(y_pred, y_true):
    return (y_pred.argmax(1) == y_true.argmax(1)).mean()


def set_training(model, mode=True):
    for p in model.parameters():
        p.requires_grad = mode
    model.train(mode)
    return model


if __name__ == '__main__':
    
    #save_folder = os.path.join(os.path.dirname(), save_folder)
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    # print('Start')
    
    random_state = np.random.randint(0, 100)  # 0から100までのランダムな整数を生成
    with open(os.path.join(save_folder, "random_state.txt"), "w") as f:
        f.write(f"random_state: {random_state}\n")
    
    # DATA.
    # set_of_train_loader,set_of_valid_loader = KFold_load_dataset_v2(data_files, batch_size,0.33)
    # print(set_of_train_loader.size())
    train_loader, valid_loader = load_dataset_v2(data_files[0:17], batch_size,random_state=random_state,split_size=0.3) 
    # for i,(train_loader,valid_loader) in enumerate(zip(set_of_train_loader,set_of_valid_loader)):
        
    dataloader = {'train': train_loader, 'valid': valid_loader, }

    #print(train_loader.shape)

    # MODEL.(list化)
    graph_args = {'strategy': 'spatial'}
    ## for sensor data
    # model = BiLSTM(input_size=15,hidden_size=64,num_layers=1,dropout_prob=0.3,num_classes=11,feature="mean").to(device)
    model = CNN_BiLSTM(hidden_size=32,num_layers=1,dropout_prob=0.3,num_classes=11,feature="mean").to(device)
    # model = TwoStreamSpatialTemporalGraph(graph_args, 11).to(device)
    n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"number of params: {n_parameters}")
    # model = TwoStreamSpatialTemporalGraph(graph_args, 2).to(device)
    # graph_args = {'strategy': 'uniform'}
    #graph_args = {'strategy': 'distance'}
    # print("err")
    #model = TwoStreamSpatialTemporalGraph(graph_args, num_class).to(device)
    #model = TwoStreamSpatialTemporalGraph(graph_args, 8).to(device)
    optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)


    # losser = torch.nn.BCELoss() #fall or no_fall
    losser = torch.nn.CrossEntropyLoss()

    # TRAINING.
    loss_list = {'train': [], 'valid': []}
    accu_list = {'train': [], 'valid': []}
    best_acc = -1
    for e in range(epochs):
        print('Epoch {}/{}'.format(e, epochs - 1))
        for phase in ['train', 'valid']:
            if phase == 'train':
                model = set_training(model, True)
            else:
                model = set_training(model, False)

            run_loss = 0.0
            run_accu = 0.0
            with tqdm(dataloader[phase], desc=phase) as iterator:
                for info in iterator:
                    # print(len(info))
                    pts, ser, lbs = info
                    # print(torch.any(torch.isnan(pts)))
                    # Create motion input by distance of points (x, y) of the same node
                    # in two frames.
                    #print("err")
                    mot = pts[:, :2, 1:, :] - pts[:, :2, :-1, :]

                    mot = mot.to(device)
                    pts = pts.to(device)
                    ser = ser.to(device)
                    # print(ser.size())
                    # torch.Size([32, 30, 15])
                    lbs = lbs.to(device)

                    #print(pts.size())torch.Size([32, 3, 30, 14])
                    #print(mot.size())torch.Size([32, 2, 29, 14])

                    # Forward.
                    # out = model((pts, mot))#タプル型
                    out = model(ser) # for sensor data

                    # out = stgcn_sensor_model((pts, mot,ser))

                    #out = model(mot)#タプル型
                    # print(lbs)

                    # print(out)
                    loss = losser(out, lbs)
                    #print("err")
                    if phase == 'train':
                        # Backward.
                        model.zero_grad()
                        loss.backward()
                        optimizer.step()

                    run_loss += loss.item()
                    accu = accuracy_batch(out.detach().cpu().numpy(),
                                          lbs.detach().cpu().numpy())
                    run_accu += accu

                    iterator.set_postfix_str(' loss: {:.4f}, accu: {:.4f}'.format(
                        loss.item(), accu))
                    iterator.update()
                    #break
            loss_list[phase].append(run_loss / len(iterator))
            accu_list[phase].append(run_accu / len(iterator))
            #print(accu_list)
            #print(torch.max(accu_list))
        if(best_acc < accu_list['valid'][-1]):
            best_acc = accu_list['valid'][-1]
            best_model = copy.deepcopy(model)
            torch.save(model.state_dict(), os.path.join(save_folder, 'tsstg-model_best.pth'))
            #break

        print('Summary epoch:\n - Train loss: {:.4f}, accu: {:.4f}\n - Valid loss:'
        ' {:.4f}, accu: {:.4f}'.format(loss_list['train'][-1], accu_list['train'][-1],
        loss_list['valid'][-1], accu_list['valid'][-1]))
#             print(best_acc)
#             plt.figure()
#             plt.plot(list(range(len(loss_list['train']))),loss_list['train'],label="train_loss")
#             plt.plot(list(range(len(loss_list['valid']))),loss_list['valid'],label="valid_loss")
#             plt.legend()
#             plt.savefig(os.path.join(save_folder, f'musa-model_of_loss_report_ur.png'))
#             plt.clf()
#             plt.close()

#             plt.figure()
#             plt.plot(list(range(len(accu_list['train']))),accu_list['train'],label="train_acc")
#             plt.plot(list(range(len(accu_list['valid']))),accu_list['valid'],label="valid_acc")
#             plt.legend()
#             plt.savefig(os.path.join(save_folder, f'musa-model_of_acc_report.png'))
#             plt.clf()
#             plt.close()
#             df = pd.DataFrame([loss_list['train'],loss_list['valid']]) 
#             df.to_csv(os.path.join(save_folder, f'musa-model_of_acc_report.csv'))

#             df = pd.DataFrame([accu_list['train'],accu_list['valid']])
#             df.to_csv(os.path.join(save_folder, f'musa-model_of_accu_report.csv'))
#             torch.save(best_model.state_dict(), os.path.join(save_folder, f'musa-model_of_{best_acc:.4f}_UR.pth'))
        print(best_acc)
        plt.figure()
        plt.plot(list(range(len(loss_list['train']))),loss_list['train'],label="train_loss")
        plt.plot(list(range(len(loss_list['valid']))),loss_list['valid'],label="valid_loss")
        plt.legend()
        plt.savefig(os.path.join(save_folder, f'sensor-model_of_loss_report.png'))
        plt.clf()
        plt.close()

        plt.figure()
        plt.plot(list(range(len(accu_list['train']))),accu_list['train'],label="train_acc")
        plt.plot(list(range(len(accu_list['valid']))),accu_list['valid'],label="valid_acc")
        plt.legend()
        plt.savefig(os.path.join(save_folder, f'sensor-model_of_acc_report.png'))
        plt.clf()
        plt.close()
        df = pd.DataFrame([loss_list['train'],loss_list['valid']]) 
        df.to_csv(os.path.join(save_folder, f'sensor-model_of_loss_report.csv'))

        df = pd.DataFrame([accu_list['train'],accu_list['valid']])
        df.to_csv(os.path.join(save_folder, f'sensor-model_of_acc_report.csv'))

        # SAVE.
        '''
        if(best_acc < accu_list['valid'][-1]):
            best_acc = accu_list['valid'][-1]
            torch.save(model.state_dict(), os.path.join(save_folder, 'tsstg-model_best.pth'))
            '''
        '''
        plot_graphs(list(loss_list.values()), list(loss_list.keys()),
                        'Last Train: {:.2f}, Valid: {:.2f}'.format(
                            loss_list['train'][-1], loss_list['valid'][-1]
                        ), 'Loss', xlim=[0, epochs],
                        save=os.path.join(save_folder, 'loss_graph.png'))
        plot_graphs(list(accu_list.values()), list(accu_list.keys()),
                        'Last Train: {:.2f}, Valid: {:.2f}'.format(
                            accu_list['train'][-1], accu_list['valid'][-1]
                        ), 'Accu', xlim=[0, epochs],
                        save=os.path.join(save_folder, 'accu_graph.png'))
        '''
            #break

    # del train_loader, valid_loader

    #model.load_state_dict(torch.load(os.path.join(save_folder, 'tsstg-model.pth',map_location=torch.device('cpu'))))
    model.load_state_dict(torch.load(os.path.join(save_folder, 'tsstg-model_best.pth')))
    # EVALUATION.
    #URのときは全部コメント
    # model = set_training(model, False)
    # data_file = data_files[1]
    # eval_loader, _ = load_dataset([data_file], 32)

    print('Evaluation.')
    run_loss = 0.0
    run_accu = 0.0
    y_preds = []
    y_trues = []
    model = best_model
    #with tqdm(eval_loader, desc='eval') as iterator:
    #URFD
    with tqdm(dataloader[phase], desc='eval') as iterator:
        for pts,ser, lbs in iterator:
            # print(lbs)
            mot = pts[:, :2, 1:, :] - pts[:, :2, :-1, :]
            mot = mot.to(device)
            pts = pts.to(device)
            ser = ser.to(device)
            lbs = lbs.to(device)

            # out = model((pts, mot)) # for skeleton data
            out = model(ser) # for sensor data
            #out = model(mot)################
            loss = losser(out, lbs)

            run_loss += loss.item()
            accu = accuracy_batch(out.detach().cpu().numpy(),
                                  lbs.detach().cpu().numpy())
            run_accu += accu

            y_preds.extend(out.argmax(1).detach().cpu().numpy())
            y_trues.extend(lbs.argmax(1).cpu().numpy())

            iterator.set_postfix_str('loss: {:.4f}, accu: {:.4f}'.format(
                loss.item(), accu))
            iterator.update()

    run_loss = run_loss / len(iterator)
    run_accu = run_accu / len(iterator)
    print('Eval Loss {:.4f}, Accu: {:.4f}'.format(run_loss, run_accu))
    print('Precision:', precision_score(y_trues, y_preds,average='micro'))
    print('Recall:', recall_score(y_trues, y_preds,average='micro'))
    print('F1-score:', f1_score(y_trues, y_preds,average='micro'))
#     tn, fp, fn, tp = metrics.confusion_matrix(y_trues, y_preds).ravel()
#     specificity  = tn / (tn + fp)
#     print('Specificity:', specificity)
    print(classification_report(y_trues, y_preds,digits=5))
    report=classification_report(y_trues,  y_preds, digits=5,output_dict=True)
    report_df = pd.DataFrame(report).T
    report_df.to_csv(os.path.join(save_folder, f'sensor-model_of_{run_accu:.4f}_report.csv'))

    cmx_data = confusion_matrix(y_trues, y_preds)

    df_cmx = pd.DataFrame(cmx_data)

    plt.figure()
    sns.heatmap(df_cmx, annot=True)
    plt.savefig(os.path.join(save_folder, f'sensor-model_ofs_confution_matrix.png'))


using cuda device
Random State Used for Split: 53


/home/egawa/anaconda3/envs/pt113/lib/python3.10/site-packages/torch/nn/modules/rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


number of params: 67195
Epoch 0/99


valid: 100%|██████████| 3769/3769 [00:56<00:00, 66.48it/s,  loss: 2.8703, accu: 0.0000]


Summary epoch:
 - Train loss: 0.4833, accu: 0.9498
 - Valid loss: 0.4501, accu: 0.9653
0.9652925179092597
Epoch 1/99


...

valid: 100%|██████████| 3769/3769 [00:56<00:00, 66.18it/s,  loss: 2.4173, accu: 0.0000]


Summary epoch:
 - Train loss: 0.4294, accu: 0.9817
 - Valid loss: 0.4436, accu: 0.9701
0.9701014858052533
Epoch 3/99


valid: 100%|██████████| 3769/3769 [00:58<00:00, 64.66it/s,  loss: 2.0472, accu: 0.0000]


Summary epoch:
 - Train loss: 0.4237, accu: 0.9850
 - Valid loss: 0.4402, accu: 0.9704
0.9703585168479703
Epoch 4/99


train:  55%|█████▌    | 5069/9196 [01:45<01:27, 47.36it/s,  loss: 0.4156, accu: 1.0000]

In [9]:
def build_dataloader(data_files, batch_size, phase="train"):
    videos=[]
    features, sensors, labels = [], [], []
    
    # データの読み込み
    for fil in data_files:
        with open(fil, 'rb') as f:
            vid, fts, sr, lbs = pickle.load(f)
            videos += vid
            features.append(fts)
            sensors.append(sr)
            labels.append(lbs)
        del fts, lbs, sr
    
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    sensors = np.concatenate(sensors, axis=0)
    
    labels = labels.astype(np.float32)  # データ型の変換

    data_sample = [(feature,sensor) for feature,sensor in zip(features,sensors)]
    dataset = Skeleton_Sensor_Dataset(data_sample, labels)
    dataloader = data.DataLoader(dataset, 
                                 batch_size, 
                                 shuffle=True if phase =="train" else False,
                                 drop_last=True if phase=="train" else False
                                )
    return dataloader

In [ ]:
#run the code
import os
import time
import torch
import pickle
import numpy as np
import torch.nn.functional as F
from shutil import copyfile
from tqdm import tqdm
from torch.utils import data
from torch.optim.adadelta import Adadelta
from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn import metrics

from skeleton_transformer import SkeletonTransformer

import copy
import seaborn as sns
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt

torch.autograd.set_detect_anomaly(False)
#二分割できる（訓練用とテスト用）

#from pose_utils import motions_map
import sys
sys.path.append("../")
# from Actionsrecognition.Models import *

from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
    KFold,
)

import pandas as pd
save_folder = 'saved/GSTCAN_HAR_newKfold_sensor(Lstm)'

#device = 'cuda'
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("using", device, "device")

epochs = 100
batch_size = 32 #32

# DATA FILES.
# Should be in format of
#  inputs: (N_samples, time_steps, graph_node, channels),
#  labels: (N_samples, num_class)
#   and do some of normalizations on it. Default data create from:
#       Data.create_dataset_(1-3).py
# where
#   time_steps: Number of frame input sequence, Default: 30
#   graph_node: Number of node in skeleton, Default: 14
#   channels: Inputs data (x, y and scores), Default: 3
#   num_class: Number of pose class to train, Default: 7

data_files = ['../Data_fall2/har30_1_sensor_new-set(labelXscrw).pkl',
              '../Data_fall2/har30_2_sensor_new-set(labelXscrw).pkl',
              '../Data_fall2/har30_3_sensor_new-set(labelXscrw).pkl',
              '../Data_fall2/har30_4_sensor_new-set(labelXscrw).pkl',
              # '../Data_fall2/har30_5_sensor_new-set(labelXscrw).pkl',]
              '../Data_fall2/har30_6_sensor_new-set(labelXscrw).pkl',
              '../Data_fall2/har30_7_sensor_new-set(labelXscrw).pkl',
              '../Data_fall2/har30_8_sensor_new-set(labelXscrw).pkl',
              # '../Data_fall2/har30_9_sensor_new-set(labelXscrw).pkl',]
              '../Data_fall2/har30_10_sensor_new-set(labelXscrw).pkl',
              '../Data_fall2/har30_11_sensor_new-set(labelXscrw).pkl',
              '../Data_fall2/har30_12_sensor_new-set(labelXscrw).pkl',
              '../Data_fall2/har30_13_sensor_new-set(labelXscrw).pkl',
              '../Data_fall2/har30_14_sensor_new-set(labelXscrw).pkl',
              '../Data_fall2/har30_15_sensor_new-set(labelXscrw).pkl',
              '../Data_fall2/har30_16_sensor_new-set(labelXscrw).pkl',
              '../Data_fall2/har30_17_sensor_new-set(labelXscrw).pkl']

class_names = ['Falling_forwards_hands','Falling_forwards_knees','Falling_backwards','Falling_sidewards',
              'Falling_sitting','Walking','Standing','Sitting','Picking','Jumping','Laying']
# num_class = len(class_names)

def k_fold_cross_subject_HAR_UP(data_files):
    f_fold = {}
    data_files = np.array(data_files)
    
    for i,(train_idx,test_idx) in enumerate(KFold(n_splits=5,shuffle=True,random_state=42).split(data_files)):
        train_pkl, test_pkl = data_files[train_idx], data_files[test_idx]
        
        train_subset_pkl, valid_subset_pkl = train_test_split(train_pkl,test_size=3,shuffle=True,random_state=42)
        
        f_fold[f"{i+1}_fold"]={
            'train':train_subset_pkl,
            'valid':valid_subset_pkl,
            'test':test_pkl,
        }
        
    return f_fold

# f_fold["1_fold"] = {"train":['../Data_fall2/har30_1_sensor_new-set(labelXscrw).pkl','../Data_fall2/har30_2_sensor_new-set(labelXscrw).pkl''../Data_fall2/har30_3_sensor_new-set(labelXscrw).pkl',...],
#                     "valid":['../Data_fall2/har30_7_sensor_new-set(labelXscrw).pkl','../Data_fall2/har30_8_sensor_new-set(labelXscrw).pkl','../Data_fall2/har30_9_sensor_new-set(labelXscrw).pkl',...],
#                     "test":['../Data_fall2/har30_10_sensor_new-set(labelXscrw).pkl','../Data_fall2/har30_11_sensor_new-set(labelXscrw).pkl','../Data_fall2/har30_12_sensor_new-set(labelXscrw).pkl',...]
#                    }
                    
        


def accuracy_batch(y_pred, y_true):
    return (y_pred.argmax(1) == y_true.argmax(1)).mean()


def set_training(model, mode=True):
    for p in model.parameters():
        p.requires_grad = mode
    model.train(mode)
    return model


if __name__ == '__main__':
    
    #save_folder = os.path.join(os.path.dirname(), save_folder)
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    # print('Start')

    # DATA.
    # set_of_train_loader,set_of_valid_loader = KFold_load_dataset_v2(data_files, batch_size,0.33)
    # print(set_of_train_loader.size())
    # train_loader, valid_loader = load_dataset_v2(data_files[0:1], batch_size,split_size=0.2) 
    # for i,(train_loader,valid_loader) in enumerate(zip(set_of_train_loader,set_of_valid_loader)):
    
    kfold_dict = k_fold_cross_subject_HAR_UP(data_files)  
    for i in kfold_dict.keys():
        
        dataloader = dict()
        for phase in ['train','valid','test']:
            pkl_files = kfold_dict[i][phase]
            dataloader[phase] = build_dataloader(pkl_files,batch_size, phase=phase)


        # MODEL.(list化)
        graph_args = {'strategy': 'spatial'}
        ## for sensor data
        model = BiLSTM(input_size=15,hidden_size=32,num_layers=1,dropout_prob=0.3,num_classes=11,feature="mean").to(device)
        # model = CNN_BiLSTM(hidden_size=32,num_layers=1,dropout_prob=0.3,num_classes=11,feature="mean").to(device)
        # model = SkeletonTransformer(in_channels=3,n_joints=14,seq_len=30,num_classes=11,embedding_dim=32,n_block=6,head_dim=16,n_heads=8).to(device)
        # model = TwoStreamSpatialTemporalGraph(graph_args, 11).to(device)
        # model = E
        n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"number of params: {n_parameters}")
        # model = TwoStreamSpatialTemporalGraph(graph_args, 2).to(device)
        # graph_args = {'strategy': 'uniform'}
        #graph_args = {'strategy': 'distance'}
        # print("err")
        #model = TwoStreamSpatialTemporalGraph(graph_args, num_class).to(device)
        #model = TwoStreamSpatialTemporalGraph(graph_args, 8).to(device)
        optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)


        # losser = torch.nn.BCELoss() #fall or no_fall
        losser = torch.nn.CrossEntropyLoss()

        # TRAINING.
        loss_list = {'train': [], 'valid': []}
        accu_list = {'train': [], 'valid': []}
        best_acc = -1
        for e in range(epochs):
            print('Epoch {}/{}'.format(e, epochs - 1))
            for phase in ['train', 'valid']:
                if phase == 'train':
                    model = set_training(model, True)
                else:
                    model = set_training(model, False)

                run_loss = 0.0
                run_accu = 0.0
                with tqdm(dataloader[phase], desc=phase) as iterator:
                    for info in iterator:
                        # print(len(info))
                        pts, ser, lbs = info
                        # print(torch.any(torch.isnan(pts)))
                        # Create motion input by distance of points (x, y) of the same node
                        # in two frames.
                        #print("err")
                        mot = pts[:, :2, 1:, :] - pts[:, :2, :-1, :]

                        mot = mot.to(device)
                        pts = pts.to(device)
                        ser = ser.to(device)
                        # print(ser.size())
                        # torch.Size([32, 30, 15])
                        lbs = lbs.to(device)

                        #print(pts.size())torch.Size([32, 3, 30, 14])
                        #print(mot.size())torch.Size([32, 2, 29, 14])

                        # Forward.
                        # out = model((pts, mot))#タプル型
                        out = model(ser) # for sensor data
                        # pts = pts.unsqueeze(-1)
                        # out = model(pts)
                        # out = model((pts, mot, ser))
                        # out = stgcn_sensor_model((pts, mot,ser))

                        #out = model(mot)#タプル型
                        # print(lbs)

                        # print(out)
                        loss = losser(out, lbs)
                        #print("err")
                        if phase == 'train':
                            # Backward.
                            model.zero_grad()
                            loss.backward()
                            optimizer.step()

                        run_loss += loss.item()
                        accu = accuracy_batch(out.detach().cpu().numpy(),
                                              lbs.detach().cpu().numpy())
                        run_accu += accu

                        iterator.set_postfix_str(' loss: {:.4f}, accu: {:.4f}'.format(
                            loss.item(), accu))
                        iterator.update()
                        #break
                loss_list[phase].append(run_loss / len(iterator))
                accu_list[phase].append(run_accu / len(iterator))
                #print(accu_list)
                #print(torch.max(accu_list))
            if(best_acc < accu_list['valid'][-1]):
                best_acc = accu_list['valid'][-1]
                best_model = copy.deepcopy(model)
                torch.save(model.state_dict(), os.path.join(save_folder, 'tsstg-model_best.pth'))
                #break

            print('Summary epoch:\n - Train loss: {:.4f}, accu: {:.4f}\n - Valid loss:'
            ' {:.4f}, accu: {:.4f}'.format(loss_list['train'][-1], accu_list['train'][-1],
            loss_list['valid'][-1], accu_list['valid'][-1]))
#             print(best_acc)
#             plt.figure()
#             plt.plot(list(range(len(loss_list['train']))),loss_list['train'],label="train_loss")
#             plt.plot(list(range(len(loss_list['valid']))),loss_list['valid'],label="valid_loss")
#             plt.legend()
#             plt.savefig(os.path.join(save_folder, f'musa-model_of_loss_report_ur.png'))
#             plt.clf()
#             plt.close()

#             plt.figure()
#             plt.plot(list(range(len(accu_list['train']))),accu_list['train'],label="train_acc")
#             plt.plot(list(range(len(accu_list['valid']))),accu_list['valid'],label="valid_acc")
#             plt.legend()
#             plt.savefig(os.path.join(save_folder, f'musa-model_of_acc_report.png'))
#             plt.clf()
#             plt.close()
#             df = pd.DataFrame([loss_list['train'],loss_list['valid']]) 
#             df.to_csv(os.path.join(save_folder, f'musa-model_of_acc_report.csv'))

#             df = pd.DataFrame([accu_list['train'],accu_list['valid']])
#             df.to_csv(os.path.join(save_folder, f'musa-model_of_accu_report.csv'))
#             torch.save(best_model.state_dict(), os.path.join(save_folder, f'musa-model_of_{best_acc:.4f}_UR.pth'))
            print(best_acc)
            plt.figure()
            plt.plot(list(range(len(loss_list['train']))),loss_list['train'],label="train_loss")
            plt.plot(list(range(len(loss_list['valid']))),loss_list['valid'],label="valid_loss")
            plt.legend()
            plt.savefig(os.path.join(save_folder, f'stgcn-model_{i }of{len(kfold_dict)}_loss_report.png'))
            plt.clf()
            plt.close()

            plt.figure()
            plt.plot(list(range(len(accu_list['train']))),accu_list['train'],label="train_acc")
            plt.plot(list(range(len(accu_list['valid']))),accu_list['valid'],label="valid_acc")
            plt.legend()
            plt.savefig(os.path.join(save_folder, f'stgcn-model_{i}of{len(kfold_dict)}_acc_report.png'))
            plt.clf()
            plt.close()
            df = pd.DataFrame([loss_list['train'],loss_list['valid']]) 
            df.to_csv(os.path.join(save_folder, f'stgcn-model_{i}of{len(kfold_dict)}_acc_report.csv'))

            df = pd.DataFrame([accu_list['train'],accu_list['valid']])
            df.to_csv(os.path.join(save_folder, f'stgcn-model_{i}of{len(kfold_dict)}_accu_report.csv'))

            # SAVE.
            '''
            if(best_acc < accu_list['valid'][-1]):
                best_acc = accu_list['valid'][-1]
                torch.save(model.state_dict(), os.path.join(save_folder, 'tsstg-model_best.pth'))
                '''
            '''
            plot_graphs(list(loss_list.values()), list(loss_list.keys()),
                            'Last Train: {:.2f}, Valid: {:.2f}'.format(
                                loss_list['train'][-1], loss_list['valid'][-1]
                            ), 'Loss', xlim=[0, epochs],
                            save=os.path.join(save_folder, 'loss_graph.png'))
            plot_graphs(list(accu_list.values()), list(accu_list.keys()),
                            'Last Train: {:.2f}, Valid: {:.2f}'.format(
                                accu_list['train'][-1], accu_list['valid'][-1]
                            ), 'Accu', xlim=[0, epochs],
                            save=os.path.join(save_folder, 'accu_graph.png'))
            '''
                #break

        # del train_loader, valid_loader

        #model.load_state_dict(torch.load(os.path.join(save_folder, 'tsstg-model.pth',map_location=torch.device('cpu'))))
        model.load_state_dict(torch.load(os.path.join(save_folder, 'tsstg-model_best.pth')))
        # EVALUATION.
        #URのときは全部コメント
        # model = set_training(model, False)
        # data_file = data_files[1]
        # eval_loader, _ = load_dataset([data_file], 32)

        print('Evaluation.')
        run_loss = 0.0
        run_accu = 0.0
        y_preds = []
        y_trues = []
        model = best_model
        #with tqdm(eval_loader, desc='eval') as iterator:
        #URFD
        with tqdm(dataloader['test'], desc='eval') as iterator:
            for pts,ser, lbs in iterator:
                # print(lbs)
                mot = pts[:, :2, 1:, :] - pts[:, :2, :-1, :]
                mot = mot.to(device)
                pts = pts.to(device)
                ser = ser.to(device)
                lbs = lbs.to(device)

                # out = model((pts, mot)) # for skeleton data
                out = model(ser) # for sensor data
                # pts = pts.unsqueeze(-1)
                # out = model(pts)
                # out = model((pts, mot, ser))
                # out = model(mot)################
                loss = losser(out, lbs)

                run_loss += loss.item()
                accu = accuracy_batch(out.detach().cpu().numpy(),
                                      lbs.detach().cpu().numpy())
                run_accu += accu

                y_preds.extend(out.argmax(1).detach().cpu().numpy())
                y_trues.extend(lbs.argmax(1).cpu().numpy())

                iterator.set_postfix_str('loss: {:.4f}, accu: {:.4f}'.format(
                    loss.item(), accu))
                iterator.update()

        run_loss = run_loss / len(iterator)
        run_accu = run_accu / len(iterator)
        print('Eval Loss {:.4f}, Accu: {:.4f}'.format(run_loss, run_accu))
        print('Precision:', precision_score(y_trues, y_preds,average='micro'))
        print('Recall:', recall_score(y_trues, y_preds,average='micro'))
        print('F1-score:', f1_score(y_trues, y_preds,average='micro'))
    #     tn, fp, fn, tp = metrics.confusion_matrix(y_trues, y_preds).ravel()
    #     specificity  = tn / (tn + fp)
    #     print('Specificity:', specificity)
        print(classification_report(y_trues, y_preds,digits=5))
        report=classification_report(y_trues,  y_preds, digits=5,output_dict=True)
        report_df = pd.DataFrame(report).T
        report_df.to_csv(os.path.join(save_folder, f'GSTCAN_{i}of{len(kfold_dict)}_{best_acc:.4f}_report.csv'))

        cmx_data = confusion_matrix(y_trues, y_preds)

        df_cmx = pd.DataFrame(cmx_data)

        plt.figure()
        sns.heatmap(df_cmx, annot=True)
        plt.savefig(os.path.join(save_folder, f'GSTCAN_{i}of{len(kfold_dict)}_confution_matrix.png'))

using cuda device
number of params: 14483
Epoch 0/99


valid: 100%|██████████| 2505/2505 [00:15<00:00, 157.14it/s,  loss: 2.6930, accu: 0.0000]


Summary epoch:
 - Train loss: 0.4646, accu: 0.9640
 - Valid loss: 2.7337, accu: 0.1962
0.1962450099800399
Epoch 1/99


valid: 100%|██████████| 2505/2505 [00:15<00:00, 162.28it/s,  loss: 2.3618, accu: 0.0000]


Summary epoch:
 - Train loss: 0.4135, accu: 0.9925
 - Valid loss: 2.5022, accu: 0.2063
0.2063498003992016
Epoch 2/99


...

valid: 100%|██████████| 2505/2505 [00:15<00:00, 165.10it/s,  loss: 2.4803, accu: 0.0000]


Summary epoch:
 - Train loss: 0.3985, accu: 0.9984
 - Valid loss: 2.3110, accu: 0.2367
0.23668912175648701
Epoch 6/99


valid: 100%|██████████| 2505/2505 [00:22<00:00, 110.78it/s,  loss: 2.7889, accu: 0.0000]


Summary epoch:
 - Train loss: 0.3975, accu: 0.9986
 - Valid loss: 2.2811, accu: 0.2357
0.23668912175648701
Epoch 7/99


train:   1%|▏         | 99/7672 [00:01<01:22, 92.10it/s,  loss: 0.4045, accu: 1.0000] 

In [ ]:
print(phase)Classification_Module

In [ ]:

train_loader, _ = load_dataset(data_files[0:1], batch_size) #batch_size = 32

valid_loader, train_loader_ = load_dataset(data_files[:1], batch_size, 0.2)
#print("err")
train_loader = data.DataLoader(data.ConcatDataset([train_loader.dataset, train_loader_.dataset]),
                               batch_size, shuffle=True)
dataloader = {'train': train_loader, 'valid': valid_loader}

In [ ]:
 with tqdm(dataloader[phase], desc='eval') as iterator:
        for pts, lbs in iterator:
            mot = pts[:, :2, 1:, :] - pts[:, :2, :-1, :]
            print(mot.shape)
            print(lbs)
            break


In [ ]:
import os.path

print("__file__:{}".format(os.path.dirname(__file__)))
print("dirname :{}".format(os.path.dirname(__file__)))
print("basename:{}".format(os.path.basename(__file__)))
print("files   :{}".format(os.listdir(os.path.dirname(__file__))))